In [1]:
import pandas as pd
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import GRU
from keras.models import Sequential, load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import copy



In [ ]:
from keras.layers.recurrent import GRU
from keras.models import Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Concatenate,Dropout
import copy
import numpy as np

true_stand_data=np.load(r'.//data3//true_array.npy')
false_stand_data=np.load(r'.//data3//false_array.npy')

sample_num={'true_num':true_stand_data.shape[0],'false_num':false_stand_data.shape[0]}
class_weight = {
                0: (1 / sample_num['false_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2,
                1: (1 / sample_num['true_num'] * (sample_num['true_num'] + sample_num['false_num'])) / 2
                }


true_sample_channel_1 = []
true_sample_channel_2 = []
true_sample_channel_3 = []

false_sample_channel_1 = []
false_sample_channel_2 = []
false_sample_channel_3 = []
for sample_number in range(true_stand_data.shape[0]):
    true_sample_channel_1.append(copy.deepcopy(true_stand_data[sample_number,:,:,0]))
    true_sample_channel_2.append(copy.deepcopy(true_stand_data[sample_number,:,:,1]))
    true_sample_channel_3.append(copy.deepcopy(true_stand_data[sample_number,:,:,2]))
#     true_sample.append(copy.deepcopy(np.concatenate((true_stand_data[sample_number,:,:,0], true_stand_data[sample_number,:,:,1],true_stand_data[sample_number,:,:,2]),axis=1)))

for sample_number in range(false_stand_data.shape[0]):
    false_sample_channel_1.append(copy.deepcopy(false_stand_data[sample_number,:,:,0]))
    false_sample_channel_2.append(copy.deepcopy(false_stand_data[sample_number,:,:,1]))
    false_sample_channel_3.append(copy.deepcopy(false_stand_data[sample_number,:,:,2])) 

combin_train_data1 = np.concatenate((true_sample_channel_1,false_sample_channel_1),axis=0)
combin_train_data2 = np.concatenate((true_sample_channel_2,false_sample_channel_2),axis=0)
combin_train_data3 = np.concatenate((true_sample_channel_3,false_sample_channel_3),axis=0)

combin_target_data= np.concatenate((np.ones((sample_num['true_num'],1)),np.zeros((sample_num['false_num'],1))),axis=0)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
combin_target_data=ohe.fit_transform(combin_target_data).toarray()



# Model 1
in1 = Input(shape=(110,7))
x_left = GRU(units=7,return_sequences=True,input_shape=(110,21))(in1)
x_left = GRU(units=21,return_sequences=True)(x_left)
x_left = GRU(units=64,return_sequences=True)(x_left)
x_left = GRU(units=192,return_sequences=True)(x_left)
x_left = GRU(units=576,return_sequences=True)(x_left)
x_left = GRU(units=1024,return_sequences=True)(x_left)
x_left = GRU(units=256)(x_left)
x_left = Dropout(0.2)(x_left)
x_left = Dense(32,activation='relu',input_shape=(256,))(x_left)

# Model 2
in2 = Input(shape=(110,7))
x_right = GRU(units=14,return_sequences=True,input_shape=(110,21))(in2)

x_right = GRU(units=42,return_sequences=True)(x_right)
x_right = GRU(units=126,return_sequences=True)(x_right)
x_right = GRU(units=378,return_sequences=True)(x_right)
x_right = GRU(units=256)(x_right)
x_right = Dropout(0.2)(x_right)
x_right = Dense(32,activation='relu')(x_right)

# Model 2
in3 = Input(shape=(110,7))
x_middle = GRU(units=14,return_sequences=True,input_shape=(110,21))(in2)
x_middle = GRU(units=42,return_sequences=True)(x_middle)
x_middle = GRU(units=126,return_sequences=True)(x_middle)
x_middle = GRU(units=378,return_sequences=True)(x_middle)
x_middle = GRU(units=256)(x_middle)
x_middle = Dropout(0.2)(x_middle)
x_middle = Dense(32,activation='relu')(x_middle)

# Model Final
model_final_concat = Concatenate(axis=-1)([x_left,x_middle,x_right])
model_final_dense_1 = Dense(2, activation='softmax')(model_final_concat)

model = Model(inputs=[in1, in2, in3], outputs=model_final_dense_1)
print(model.summary())

callbacks_list = [
    
    keras.callbacks.EarlyStopping(
        monitor='accuracy', 
        patience=3 
    ),
    # 保存模型
    keras.callbacks.ModelCheckpoint(
    filepath = 'GRU_predict.h5', 
    monitor='loss', 
    save_best_only=True) 
]


model.compile(loss='categorical_crossentropy', #continu together
              optimizer='adam',
              metrics=['accuracy']
             
             )

model.fit(x=[combin_train_data1,combin_train_data2,combin_train_data3],
          y=combin_target_data,
          batch_size=120, 
          epochs=100,
          shuffle=True,
          verbose=1,
          class_weight=class_weight,
          callbacks=callbacks_list      
         )